# Time Travel - Exercise Solutions

This notebook contains the complete solutions for all time travel exercises. Use this to check your work or understand the concepts better.

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langgraph_sdk langgraph-prebuilt

In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

## Exercise 1 Solution: Understanding Checkpoints

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_core.messages import SystemMessage, HumanMessage

# Solution: Define basic math tools
def add(a: int, b: int) -> int:
    """Add two numbers together.
    
    Args:
        a: First number
        b: Second number
    """
    return a + b

def multiply(a: int, b: int) -> int:
    """Multiply two numbers.
    
    Args:
        a: First number
        b: Second number
    """
    return a * b

def divide(a: int, b: int) -> float:
    """Divide a by b.
    
    Args:
        a: Dividend
        b: Divisor
    """
    if b == 0:
        return "Error: Cannot divide by zero"
    return a / b

# Solution: Create tools list and LLM with tools
tools = [add, multiply, divide]
llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools(tools)

# Solution: Create system message for math assistant
sys_msg = SystemMessage(content="You are a helpful math assistant. Use the available tools to perform calculations.")

# Solution: Define assistant node
def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Solution: Build the graph
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
builder.add_edge("tools", "assistant")

# Solution: Compile with memory checkpointer
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

print("Math assistant graph created successfully!")
print("This graph will automatically save checkpoints as it executes.")

## Exercise 2 Solution: Creating Your First Execution History

In [ ]:
# Solution: Run your graph with a math problem
initial_input = {"messages": [HumanMessage(content="Calculate 15 * 4 + 8")]}
thread = {"configurable": {"thread_id": "exercise_2"}}

print("Running graph and creating execution history...")
# Solution: Execute the graph and capture each step
step_count = 0
for event in graph.stream(initial_input, thread, stream_mode="values"):
    step_count += 1
    last_message = event['messages'][-1]
    msg_type = last_message.__class__.__name__
    content_preview = getattr(last_message, 'content', str(last_message))[:80]
    print(f"Step {step_count}: {msg_type} - {content_preview}...")

In [ ]:
# Solution: Examine the current state
current_state = graph.get_state(thread)
print("Current state information:")
print(f"Number of messages: {len(current_state.values['messages'])}")
print(f"Next nodes to execute: {current_state.next}")
print(f"Checkpoint ID: {current_state.config['configurable']['checkpoint_id']}")

# Solution: Get the complete execution history
all_states = [s for s in graph.get_state_history(thread)]
print(f"\nTotal checkpoints created: {len(all_states)}")
print("Each checkpoint represents a step in the execution.")

## Exercise 3 Solution: Browsing Execution History

In [ ]:
# Solution: Create a function to analyze execution history
def analyze_execution_history(graph, thread_config):
    """Analyze and display information about execution history."""
    states = [s for s in graph.get_state_history(thread_config)]
    
    print(f"=== Execution History Analysis ({len(states)} checkpoints) ===")
    
    for i, state in enumerate(reversed(states)):
        print(f"\nCheckpoint {i+1}:")
        messages = state.values.get('messages', [])
        print(f"  Messages: {len(messages)}")
        
        if messages:
            last_msg = messages[-1]
            msg_type = last_msg.__class__.__name__
            content_preview = getattr(last_msg, 'content', str(last_msg))[:60]
            print(f"  Last message: {msg_type} - {content_preview}...")
        
        print(f"  Next node: {state.next}")
        print(f"  Checkpoint ID: {state.config['configurable']['checkpoint_id'][:8]}...")
    
    return states

# Solution: Analyze your execution history
history_states = analyze_execution_history(graph, thread)

## Exercise 4 Solution: Time Travel - Replaying from a Checkpoint

In [ ]:
# Solution: Select a checkpoint to replay from
states = [s for s in graph.get_state_history(thread)]
# Choose the state with just the human message (should be second from last in reverse order)
replay_checkpoint = states[-2]  # This should be the initial state with human message

print(f"Selected checkpoint for replay:")
print(f"State: {len(replay_checkpoint.values['messages'])} messages")
print(f"Next: {replay_checkpoint.next}")
if replay_checkpoint.values['messages']:
    last_msg_content = replay_checkpoint.values['messages'][-1].content
    print(f"Last message preview: {last_msg_content[:50]}...")

# Solution: Replay execution from this checkpoint
print("\n=== REPLAYING FROM CHECKPOINT ===")
replay_step = 0
for event in graph.stream(None, replay_checkpoint.config, stream_mode="values"):
    replay_step += 1
    last_message = event['messages'][-1]
    msg_type = last_message.__class__.__name__
    content_preview = getattr(last_message, 'content', str(last_message))[:80]
    print(f"Replay Step {replay_step}: {msg_type} - {content_preview}...")

## Exercise 5 Solution: Time Travel - Forking Execution

In [ ]:
# Solution: Find the initial state with the human message
states = [s for s in graph.get_state_history(thread)]
initial_state = states[-2]  # State with just the human message

print("Original query:", initial_state.values["messages"][0].content)

# Solution: Create a modified version of the query
original_message = initial_state.values["messages"][0]
modified_query = "Calculate the area of a circle with radius 5"

# Solution: Update the state with the modified query
fork_config = graph.update_state(
    initial_state.config,
    {"messages": [HumanMessage(content=modified_query, id=original_message.id)]}
)

print(f"\nCreated fork with new query: {modified_query}")
print(f"Fork checkpoint ID: {fork_config['configurable']['checkpoint_id'][:8]}...")

In [ ]:
# Solution: Execute the forked path
print("=== EXECUTING FORKED PATH ===")
fork_step = 0
for event in graph.stream(None, fork_config, stream_mode="values"):
    fork_step += 1
    last_message = event['messages'][-1]
    msg_type = last_message.__class__.__name__
    content_preview = getattr(last_message, 'content', str(last_message))[:80]
    print(f"Fork Step {fork_step}: {msg_type} - {content_preview}...")

# Solution: Compare the two execution paths
print("\n=== COMPARISON ===")
original_final = graph.get_state(thread)
forked_final = graph.get_state(fork_config)

print(f"Original result: {original_final.values['messages'][-1].content[:100]}...")
print(f"Forked result: {forked_final.values['messages'][-1].content[:100]}...")

## Testing the Complete Solution

In [ ]:
# Test all the basic functionality works
print("=== TESTING TIME TRAVEL FUNCTIONALITY ===")

# Test 1: Basic execution
test_input = {"messages": [HumanMessage(content="What is 7 * 8?")]}
test_thread = {"configurable": {"thread_id": "functionality_test"}}

print("\n1. Testing basic execution with checkpoints...")
for event in graph.stream(test_input, test_thread, stream_mode="values"):
    pass  # Just execute to create history

# Test 2: History browsing
print("\n2. Testing history browsing...")
test_states = [s for s in graph.get_state_history(test_thread)]
print(f"Created {len(test_states)} checkpoints")

# Test 3: Replaying
print("\n3. Testing replay functionality...")
replay_state = test_states[-2]
replay_count = 0
for event in graph.stream(None, replay_state.config, stream_mode="values"):
    replay_count += 1
print(f"Replay executed {replay_count} steps")

# Test 4: Forking
print("\n4. Testing fork functionality...")
original_msg = replay_state.values["messages"][0]
test_fork = graph.update_state(
    replay_state.config,
    {"messages": [HumanMessage(content="What is 9 * 9?", id=original_msg.id)]}
)
fork_count = 0
for event in graph.stream(None, test_fork, stream_mode="values"):
    fork_count += 1
print(f"Fork executed {fork_count} steps")

print("\n✅ All time travel functionality tests passed!")